In [1]:
from transformers import * 
import numpy as np 
import pandas as pd
import torch 
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
import time 
import datetime 
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import re
import os

/opt/conda/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
checkpoint = torch.load("../input/roberta-baseline-epoch-6/RoBERTa_baseline_6")

In [3]:
test_model = RobertaForSequenceClassification.from_pretrained("../input/huggingface-roberta/roberta-base", num_labels=1) 
test_model.load_state_dict(checkpoint)
test_model.cuda() 

Some weights of the model checkpoint at ../input/huggingface-roberta/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ../input/huggingface-roberta/roberta-base and are

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [5]:
tokenizer = RobertaTokenizerFast.from_pretrained("../input/huggingface-roberta/roberta-base") 

def roberta_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer.encode_plus(
        text = sent, 
        add_special_tokens = True, 
        pad_to_max_length = False, 
        return_attention_mask = True 
    )
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] 
    if len(input_id) > 512: 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_maks[:129] + attention_mask[-383:] 
        print("Long Text!! Using Head+Tail Truncation") 
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512-len(attention_mask)) 
        
    return input_id, attention_mask

In [6]:
BATCH_SIZE = 16
NUM_EPOCHS = 10 
VALID_SPLIT = 0.1 
MAX_LEN = 512 

In [4]:
test = pd.read_csv("../input/commonlitreadabilityprize/test.csv") 
submission = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")

In [8]:
device = torch.device("cuda")

In [9]:
test_texts = test['excerpt'].values 

predictions = [] 

for text in tqdm(test_texts, position=0,leave=True): 
    input_id, attention_mask = roberta_tokenizer(text, MAX_LEN=MAX_LEN) 
    input_id = torch.tensor(input_id, dtype=int) 
    attention_mask = torch.tensor(attention_mask, dtype=int) 
    
    input_id = torch.reshape(input_id, (-1,MAX_LEN)) 
    attention_mask = torch.reshape(attention_mask, (-1,MAX_LEN)) 
    
    input_id = input_id.to(device) 
    attention_mask = attention_mask.to(device) 
    
    with torch.no_grad(): 
            outputs = test_model(input_id, 
                                 token_type_ids=None, 
                                 attention_mask=attention_mask) 
    
    yhat = outputs[0].item() 
    predictions.append(yhat)
    

100%|██████████| 7/7 [00:01<00:00,  6.19it/s]


In [10]:
submission.iloc[:,1] = predictions 

submission.to_csv("submission.csv",index=False)